In [2]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import ConversationChain
from langchain_community.vectorstores import Qdrant
from langchain.memory import ConversationSummaryMemory
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain import PromptTemplate

import qdrant_client
import os

In [3]:
qdrant_key = os.environ.get("QDRANT_KEY")
qdrant_url = "https://59f8f159-fb60-44e8-bfc4-9f35c77ca8d4.us-east4-0.gcp.cloud.qdrant.io:6333"

# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/code/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

client = qdrant_client.QdrantClient(
    qdrant_url,
    api_key=qdrant_key,
)

doc_store = Qdrant(
    client=client, collection_name="rakuten", 
    embeddings=embeddings,
)

In [4]:
conversation_openai = ConversationChain(
        # llm=ChatOpenAI(model='gpt-3.5-turbo-0613'),
        llm=ChatOpenAI(model='gpt-3.5-turbo'),
        memory=ConversationSummaryMemory(llm=ChatOpenAI(model='gpt-3.5-turbo-0613')),
        verbose=False, 
    )

conversation_anthropic = ConversationChain(
        llm=ChatAnthropic(model='claude-2.1'),
        memory=ConversationSummaryMemory(llm=ChatAnthropic(model='claude-2.1')),
        verbose=False
    )

messages = [
    {"role": "assistant", "content": "How can I help you today?"}
]

In [5]:
# Get relevant docs through vector DB

SIMILARITY_THRESHOLD = 0.83

# Create prompt
template = """ You are a virtual assistant for Rakuten and your task is to answer questions related to Rakuten which includes general information about Rakuten.

                Respond in the style of a polite helpful assistant and do not allude that you have looked up the context.

                Do not hallucinate. If you don't find an answer, you can point user to the official website here: https://www.rakuten.com/help . 

                In your response, include the following url links at the end of your response {url_links} and any other relevant URL links that you refered.

                Also, at the end of your response, ask if your response was helpful". 

                Here is some relevant context: {context}"""


def get_relevant_docs(user_input):
 
    relevant_docs = doc_store.similarity_search_with_score(user_input, k=3)
    urls = [result[0].metadata['source'] for result in relevant_docs if result[1] > SIMILARITY_THRESHOLD]
    contexts = [result[0].page_content for result in relevant_docs if result[1] > SIMILARITY_THRESHOLD]
 
    return urls, contexts
 
 
def build_system_prompt(user_input):
 
    urls, contexts = get_relevant_docs(user_input)
    
    prompt_template = PromptTemplate(
        input_variables=["url_links", "context"],
        template=template
    )
    system_prompt = prompt_template.format(url_links=urls, context=contexts)
 
    return system_prompt, urls, contexts
 
# Query the AI Model
def queryAIModel(user_input, llm="openai", return_chain=True):
 
    if llm.lower=="anthropic":
        conversation = conversation_anthropic
    else:
        conversation = conversation_openai
        
    if return_chain:
        return conversation
    
    system_prompt, _, _ = build_system_prompt(user_input)   
    
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=user_input
        ),
    ]
    output = conversation.predict(input=messages)
 
    return output

In [7]:
# Ask a question
# user_question = input("Please provide your question here :")
# result = queryAIModel(user_question, return_chain=False)

Please provide your question here : gift


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
result

"Hello! How can I assist you with information about Rakuten today? If you have any questions about the Rakuten Gift Card Shop, I'd be happy to help. Here is a link to the terms of sale for the Rakuten Gift Card Shop: [Rakuten Gift Card Shop Terms of Sale](https://www.rakuten.com/help/article/rakuten-gift-card-shop-terms-of-sale-360052462534). Additionally, feel free to explore more general information about Rakuten on their official website [here](https://www.rakuten.com/help/article/). If you are interested in donating part of your Big Fat Check to Cash Back for Change, you can find more details at this link: [Donate to Cash Back for Change](https://www.rakuten.com/help/article/donating-part-of-your-big-fat-check-to-cash-back-for-change-360058672214). Was this information helpful to you?"